# Retrieve all the relevant data from the table athletic-data in the database.
# Pre-requisite :
  1. Activity has to be of more than 2hrs.
  2. Watts and HeartRate column should not be null.

In [119]:
%reset
import pandas as pd

from src.config.LoadProperties import retrieve_data_sql
from src.repositories.PowerAndHRRepository import configs, sql_engine
year = 2021
athlete_id = 863203

"""
Read and return athletic data from a SQL database.

:return: A pandas DataFrame containing the athletic data.
"""

def get_athletic_data(id, activity_year):
    sql = retrieve_data_sql.replace("(athlete_id)",str(id)+")").replace("(year)",str(activity_year))
    return pd.read_sql(sql, sql_engine)


athletic_data_db = get_athletic_data(athlete_id, year)

# DATA PROFILING : Removing irrelevant columns not required for analysis.

In [ ]:
cols_to_ignore = configs.get("ignore-columns").data
athletic_data = athletic_data_db.drop(cols_to_ignore.split(","), axis=1).rename(columns={'watts' : 'power'})
athletic_data['distance'] = athletic_data['distance'].apply(lambda x: [float(value) for value in x])


# Check if the lengths of lists in specified columns are the same for each row


In [ ]:
def check_lengths(row):
    columns_to_check = ['distance', 'heartrate', 'power', 'time']
    lengths = [len(row[col]) for col in columns_to_check]
    return all(length == lengths[0] for length in lengths)

# Apply the function to each row and count the activity IDs with different lengths
result = athletic_data[athletic_data.apply(check_lengths, axis=1)]
count_different_lengths = len(athletic_data) - len(result)

print(f"Count of activity IDs with different lengths: {count_different_lengths}")

# NA/NONE CHECK : Checking if watts or heartrate column contains null values

In [ ]:

hr_contains_none = athletic_data['heartrate'].apply(lambda x: any(pd.isna(item) for item in x) if pd.notna(x) else False).sum()
print(f"{hr_contains_none}/{len(athletic_data['heartrate'])} entries contains NA/NONE/null in heartrate")
watts_contains_none =  athletic_data['power'].apply(lambda x: any(pd.isna(item) for item in x) if pd.notna(x) else False).sum()
print(f"{watts_contains_none}/{len(athletic_data['power'])}  entries contains NA/NONE/null in Power")
distance_contains_none =  athletic_data['distance'].apply(lambda x: any(pd.isna(item) for item in x) if pd.notna(x) else False).sum()
print(f"{distance_contains_none}/{len(athletic_data['distance'])}  entries contains NA/NONE/null in Distance")
time_contains_none =  athletic_data['time'].apply(lambda x: any(pd.isna(item) for item in x) if pd.notna(x) else False).sum()
print(f"{time_contains_none}/{len(athletic_data['time'])}  entries contains NA/NONE/null in Time")

# Cluster view of how long the NA/None/Null entries exists.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

athletic_data_with_na = pd.DataFrame(index=athletic_data['activity_id'], columns=range(len(max(athletic_data['power'], key=len))))
 

for idx, powers in zip(athletic_data['activity_id'], athletic_data['power']):
    athletic_data_with_na.loc[idx, :len(powers) - 1] = pd.Series(powers).isna().values

athletic_data_with_na = athletic_data_with_na.astype(float)

cmap = sns.color_palette(['#00FF00', '#FF0000'])

# Create a line plot with color gradient
plt.figure(figsize=(14, 8))
sns.heatmap(data=athletic_data_with_na, cmap=cmap, cbar=False, xticklabels=False) 
plt.title('Visualization of NA/None/Null Values in Power Column')
plt.xlabel('Activity Length(in seconds)')
plt.ylabel('Activity ID')
plt.show()

# Visualization of the length of NA/None/Null Values in Power Column

In [ ]:
import matplotlib.pyplot as plt
from itertools import groupby
from collections import Counter

cons_na_pow_list = athletic_data['power'].apply(lambda x: [0] + [len(list(g)) for k, g in groupby(x) if pd.isna(k)]).tolist()

def count_occurrences(na_list):
    flat_list = [min(num, 60) for sublist in na_list for num in sublist if num != 0]
    occurrences = Counter(flat_list)
    return dict(occurrences)

seq_na_occ_count = count_occurrences(cons_na_pow_list)

plt.figure(figsize=(30, 12))

plt.bar(seq_na_occ_count.keys(), seq_na_occ_count.values(), color='blue', edgecolor='black')

for key, value in seq_na_occ_count.items():
    plt.text(key, value, str(value), ha='center', va='bottom')


plt.title('Occurrence Count Histogram')
plt.xlabel('Consecutive Time(in seconds)')
plt.ylabel('Sequential NA Occurrence Count') 
plt.grid(axis='y')
plt.show()

# Interpolation : Applying Linear Interpolation only for NA/None occurring for maximum 5 seconds in the power data recorded.

In [ ]:
li_na = 5
def linear_interpolate_na(seq):
    interpolate_df = pd.DataFrame({'power': seq})

    interpolated_seq = interpolate_df.interpolate(limit=li_na)

    return interpolated_seq['power'].tolist()

interpolated_power = [linear_interpolate_na(seq) for seq in athletic_data['power']]

# Adding Index column of segment starting index


In [ ]:
athletic_data['power_ip'] = interpolated_power

athletic_data['power_ip_na_adjusted'] = athletic_data['power_ip'].apply(lambda x: [x[i] for i in range(len(x)) if pd.notna(x[i])])

#list_a = [1, 4, 5, None, None, None, 4, 5, 9, None, 4, 5, None, None, 3, 8, 10, None]
#[1, 4, 5, 4, 5, 9, 4, 5, 3, 8, 10]
#[0, 1, 2, 6, 7, 8, 10, 11, 14, 15, 16]
#[0, 3, 6, 8]
#list_ax = [None, 1, 4, 5, None, None, None, 4, 5, None, 2, 3, 9, None, None, 3, 8, 10, None]
#[1, 4, 5, 4, 5, 2, 3, 9, 3, 8, 10]
#[1, 2, 3, 7, 8, 10, 11, 12, 15, 16, 17]
#[0, 3, 5, 8]
def find_segment_start_indices(original_list):
    non_none_indices = [i for i, val in enumerate(original_list) if pd.notna(val)]

    indexes = []
    for i in range(0, len(non_none_indices)):
        if i == 0  and non_none_indices[i] >= 0:
            indexes.append(i)
        if non_none_indices[i] > non_none_indices[i-1] + 1:
            indexes.append(i)
    return indexes
 
athletic_data['segment_start_index'] = athletic_data.apply(lambda row: find_segment_start_indices(row['power_ip']), axis=1)

# NA Replacement Check

In [ ]:
import pandas as pd
from itertools import groupby

cons_na_len = 7
athletic_data['cons_na_power'] = athletic_data['power'].apply(lambda x: [0] + [len(list(g)) for k, g in groupby(x) if pd.isna(k)])

rows_with_cons_na = athletic_data[athletic_data['cons_na_power'].apply(lambda x: cons_na_len in x)]

if not rows_with_cons_na.empty:
    athletic_ids_with_na = rows_with_cons_na['activity_id'].tolist()
    print(f"Activity IDs with {cons_na_len} consecutive NAs: {athletic_ids_with_na}")
else:
    print("No rows with "+ cons_na_len + "consecutive NAs found.")


# Position of the NA in the power list

In [ ]:
activity_id_to_extract = 6399104761

selected_activity = athletic_data[athletic_data['activity_id'] == activity_id_to_extract]

orig_power_df = pd.DataFrame({'power': selected_activity['power'].iloc[0]})
power_df_ip = pd.DataFrame({'power_ip': selected_activity['power_ip'].iloc[0]})
power_ip_na_adjusted = pd.DataFrame({'power_ip_na_adjusted': selected_activity['power_ip_na_adjusted'].iloc[0]})
segment_index = pd.DataFrame({'segment_start_index': selected_activity['segment_start_index'].iloc[0]})
NA_comparison_df =  pd.concat([orig_power_df, power_df_ip, power_ip_na_adjusted, segment_index], axis=1)

# Choosing the longest continuous segments of each of the rides

In [ ]:
def find_longest_segment_indices(lst):
    if not lst:
        return [None, None]

    start_index, end_index = 0, 0
    current_start, current_end = 0, 0
    max_length = 1

    for i in range(1, len(lst)):
        if lst[i] == lst[i - 1] + 1:
            current_end = i
        else:
            current_start = i
            current_end = i

        current_length = current_end - current_start + 1

        if current_length > max_length:
            start_index, end_index = current_start, current_end
            max_length = current_length

    return [start_index, end_index]

athletic_data['longest_time_segment'] = athletic_data['time'].apply(find_longest_segment_indices)


# Calculating datapoints of all the activities

In [ ]:
get_list_length = lambda x: len(x) if isinstance(x, list) else 0

athletic_data['datapoints'] = athletic_data['power'].apply(get_list_length)

# Converting Dataframe into CSV for further analysis in MATLAB

In [ ]:
import os
import pandas as pd

df = pd.DataFrame(athletic_data)

athlete_name="Connor Swift"
athlete_id="863203"
year=2021

csv_directory = f'Strava Data/{athlete_name}'
csv_path = f'{csv_directory}/{athlete_name}_{athlete_id}_{year}.csv'

if not os.path.exists(csv_directory):
    os.makedirs(csv_directory)

df.to_csv(csv_path, index=False)


print(f"DataFrame has been successfully converted to CSV and saved at: {csv_path}")